In [1]:
import pandas as pd
import numpy as np

# Preprocessing clinical data

We do this prior to upload because of patient merging issues.

In [ ]:
from redcap_preprocessing import redcap_preprocessing

redcap_filepath = '/Users/gustaveronteix/Documents/data/OraklOnco/clinical/upload/ColonGR-PDO_DATA_2023-10-18_1504.csv'
output_dir = '/Users/gustaveronteix/Documents/data/OraklOnco/clinical/upload/single_files'
conversion_table_filepath = 'conversion_table/redcap_CRC_conversion_table.csv'
disease_type = 'CRC'

redcap_preprocessing.preprocess_redcap_data(redcap_filepath,
                                            conversion_table_filepath,
                                            output_dir,
                                            disease_type,
                                            True,)

In [2]:
from redcap_preprocessing import redcap_preprocessing

redcap_filepath = '/Users/gustaveronteix/Documents/data/OraklOnco/clinical/upload/ColonGR-PDO_DATA_2023-10-18_1504.csv'
output_dir = '/Users/gustaveronteix/Documents/data/OraklOnco/clinical/upload/single_files'
conversion_table_filepath = 'conversion_table/redcap_CRC_conversion_table.csv'
disease_type = 'CRC'

redcap_preprocessing.preprocess_redcap_data(redcap_filepath,
                                            disease_type=disease_type,
                                            output_dir=output_dir,
                                            save_as_single_file = True)

There are no PDO cell lines for record_id 53.
There are no PDO cell lines for record_id 53.
There are no PDO cell lines for record_id 53.
There are no PDO cell lines for record_id 53.
There are no PDO cell lines for record_id 53.
There are no PDO cell lines for record_id 53.


In [3]:
from redcap_preprocessing import redcap_preprocessing

redcap_filepath = '/Users/gustaveronteix/Documents/data/OraklOnco/clinical/upload/PANCREASGR-PDO_DATA_2023-10-18_1509.csv'
output_dir = '/Users/gustaveronteix/Documents/data/OraklOnco/clinical/upload/single_files'
conversion_table_filepath = 'conversion_table/redcap_PGR_conversion_table.csv'
disease_type = 'PDAC'

redcap_preprocessing.preprocess_redcap_data(redcap_filepath,
                                            disease_type=disease_type,
                                            output_dir=output_dir,
                                            save_as_single_file = True)

There are no PDO cell lines for record_id 12.
There are no PDO cell lines for record_id 12.
There are no PDO cell lines for record_id 101.
There are no PDO cell lines for record_id 101.
There are no PDO cell lines for record_id 450.
There are no PDO cell lines for record_id 450.


KeyError: "['record_id', 'cell_line_code', 'date_cell_line'] not in index"